get mass

In [25]:
%run -i 'get_mass.py'

# Get data from uniprot api

In [1]:
import retrieve_from_xml_uniprotapi as uniprotxml

In [2]:
datos = uniprotxml.get_uniprot_data('P31749')

/homes/dtg30/Desktop/group_proj/venv/src/Group-project/python_scripts/retrieve_from_xml_uniprotapi.py:17: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 17 of the file /homes/dtg30/Desktop/group_proj/venv/src/Group-project/python_scripts/retrieve_from_xml_uniprotapi.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  bs_data = BeautifulSoup(requested_xml.content)


In [3]:
uniprotxml.get_accession(datos)

'P31749'

In [26]:
import sys
del sys.modules["uniprotxml"]
del uniprotxml


KeyError: 'uniprotxml'

# Random things, tests

In [1]:
def get_uniprot_data(accession):
    ''' get data from uniprot api. The data is in xml. This uses bs4 BeautifulSoup to parse it
     for eg: https://www.ebi.ac.uk/proteins/api/coordinates?offset=0&size=100&accession=P31749'''
    text = 'https://www.ebi.ac.uk/proteins/api/coordinates?offset=0&size=100&accession={}'.format(accession)
    try:
        import requests
    except:
        'requests package not installed. Run "pip install -r requirements"'
    try:
        from bs4 import BeautifulSoup
    except:
        print(' bs4 package not installed. Run "pip install -r requirements"')
    try:
        requested_xml = requests.get(text)
    except:
        print('Problem with the following accession number: ' + accession)
    bs_data = BeautifulSoup(requested_xml.content)
    return bs_data

In [29]:
datos = get_uniprot_data('P31749')

In [4]:
def data_modif_res(bs_data):
    ''' returns a xml list of modif residues, with all information'''
    reference = bs_data.gnentries.gnentry.findAll('gncoordinate')[0]
    features = reference.findAll('feature')
    phospho_list = []
    for entry in features:
        if entry['type'] == 'modified residue': # extract modified residues
             if 'hospho' in entry.findAll('ns2:description')[0].text: # is phosphosomething ?
                 phospho_list.append(entry)
    return phospho_list

In [46]:
data_modif_res(datos)[0]


<feature type="modified residue"><ns2:location><ns2:position position="450"></ns2:position></ns2:location><ns2:description>Phosphothreonine.</ns2:description><ns2:evidence code="ECO:0000244"><ns2:dbreference id="24275569" type="PubMed"></ns2:dbreference></ns2:evidence><genomelocation><ns2:begin position="104770760"></ns2:begin><ns2:end position="104770758"></ns2:end></genomelocation></feature>

In [41]:
def modres_prot_position(bs_data):
    mod_res_list = data_modif_res(bs_data)
    for mod_res_item in mod_res_list:
        print(mod_res_item.findAll('ns2:position')[0]['position'])
        
    
# data_modif_res(datos)[0].findAll('ns2:position')[0]['position'] # position of modif residue

# modres_prot_position(datos)

450
176
473
129
308
448
126
124
474


In [44]:
def modres_prot_descr(bs_data):
    mod_res_list = data_modif_res(bs_data)
    for mod_res_item in mod_res_list:
        print(mod_res_item.findAll('ns2:description')[0].text) # type of modif

# data_modif_res(datos)[0].findAll('ns2:description')[0].text # type of modif
modres_prot_descr(datos)              

Phosphothreonine.
Phosphotyrosine; by TNK2.
Phosphoserine; by IKKE, MTOR and TBK1; alternate.
Phosphoserine; alternate.
Phosphothreonine; by IKKE, PDPK1 and TBK1.
Phosphothreonine.
Phosphoserine; alternate.
Phosphoserine.
Phosphotyrosine.


In [51]:
def modres_prot_refs(bs_data):
    mod_res_list = data_modif_res(bs_data)
    for mod_res_item in mod_res_list:
#         print(mod_res_item.findAll('ns2:dbreference')[0]['id']) # type of modif
        references = mod_res_item.findAll('ns2:dbreference') # position of modif residue
        for reference in references:
            print(reference['id'])
            print(reference['type'])        
        
        
# data_modif_res(datos)[0].findAll('ns2:description')[0].text # type of modif
modres_prot_refs(datos)  

# data_modif_res(datos)[0].findAll('ns2:dbreference')[0]['id'] # position of modif residue

24275569
PubMed
20333297
PubMed
9736715
PubMed
8978681
PubMed
23799035
PubMed
15047712
PubMed
21464307
PubMed
16266983
PubMed
15718470
PubMed
20333297
PubMed
17013611
PubMed
14761976
PubMed
20978158
PubMed
23186163
PubMed
18669648
PubMed
17081983
PubMed
15718470
PubMed
9512493
PubMed
18456494
PubMed
8978681
PubMed
20333297
PubMed
21464307
PubMed
20481595
PubMed
24275569
PubMed
18669648
PubMed
18669648
PubMed
12149249
PubMed


In [55]:
def modres_prot_genompos(bs_data):
    mod_res_list = data_modif_res(bs_data)
    for mod_res_item in mod_res_list:
        print(mod_res_item.findAll('ns2:begin')[0]['position']) # end position
        print(mod_res_item.findAll('ns2:end')[0]['position']) # end position
        
modres_prot_genompos(datos)     



104770760
104770758
104775117
104775115
104770367
104770365
104775702
104775700
104773286
104773284
104770766
104770764
104775711
104775709
104775717
104775715
104770364
104770362


# RANDOM

In [29]:
from bs4 import BeautifulSoup

x="""<foo>
   <bar>
      <type foobar="1"/>
      <type foobar="2"/>
   </bar>
</foo>"""

y=BeautifulSoup(x)
print(y.foo.bar)

print(y.foo.bar.findAll("type"))
print(y.foo.bar.findAll("type")[0]["foobar"])

<bar>
<type foobar="1"></type>
<type foobar="2"></type>
</bar>
[<type foobar="1"></type>, <type foobar="2"></type>]
1


In [1]:
from bs4 import BeautifulSoup
import requests

In [34]:
import requests
text = 'https://www.ebi.ac.uk/proteins/api/coordinates?offset=0&size=100&accession=P31749'
aa = requests.get(text)

In [154]:
data_modif_res = y.gnentries.gnentry.findAll('gncoordinate')[0].findAll('feature')
for entry in data_modif_res:
    if entry['type'] == 'modified residue': #only mod res
        if 'hospho' in entry.findAll('ns2:description')[0].text: #description is phosphosomething
            print(entry.findAll('ns2:description')[0].text) # what is mod. needs to order this
            print(entry.findAll('ns2:position')[0]['position']) # position of modification
            print(entry.findAll('ns2:evidence')[0].findAll('ns2:dbreference')[0]['id']) #needs a loop #pubmedid
            print(entry.findAll('ns2:evidence')[0].findAll('ns2:dbreference')[0]['type'])#needs a loop #pubmed

Phosphothreonine.
450
24275569
PubMed
Phosphotyrosine; by TNK2.
176
20333297
PubMed
Phosphoserine; by IKKE, MTOR and TBK1; alternate.
473
9736715
PubMed
Phosphoserine; alternate.
129
23186163
PubMed
Phosphothreonine; by IKKE, PDPK1 and TBK1.
308
15718470
PubMed
Phosphothreonine.
448
24275569
PubMed
Phosphoserine; alternate.
126
18669648
PubMed
Phosphoserine.
124
18669648
PubMed
Phosphotyrosine.
474
12149249
PubMed
